# Day 09: Marble Mania
[link](https://adventofcode.com/2018/day/9)

## Part 1: Best player score

In [1]:
import re

regex = re.compile(r'^(\d+) players; last marble is worth (\d+) points')

def parse(s):
  match = regex.match(s)
  return int(match.group(1)), int(match.group(2))

test_in = '9 players; last marble is worth 25 points'
parse(test_in)

(9, 25)

In [2]:
def score(text, debug=False):
  n_players,last_marble = parse(text)
  circle = [0]
  current_i = 0
  player_scores = [0]*n_players
  for marble in range(1,last_marble+1):
    player_i = (marble-1)%n_players # The number (starting with 0) of the player that makes the move with `marble`
    if marble%23 > 0:
      current_i = current_i+2 if current_i<len(circle)-1 else 1
      circle.insert(current_i, marble)
    else: # A winning turn for the curent player
      current_i = current_i-7 if current_i>=7 else current_i+len(circle)-7
      player_scores[player_i] += circle.pop(current_i) + marble

    if debug:
      s = ''.join(f'({str(m).rjust(2)})' if i==current_i else f' {str(m).rjust(2)} ' for i,m in enumerate(circle))
      print(f'[{player_i+1}] {s}')

  max_score = max(player_scores)
  return max_score, player_scores.index(max_score)

assert score(test_in, True) == (32,4)

[1]   0 ( 1)
[2]   0 ( 2)  1 
[3]   0   2   1 ( 3)
[4]   0 ( 4)  2   1   3 
[5]   0   4   2 ( 5)  1   3 
[6]   0   4   2   5   1 ( 6)  3 
[7]   0   4   2   5   1   6   3 ( 7)
[8]   0 ( 8)  4   2   5   1   6   3   7 
[9]   0   8   4 ( 9)  2   5   1   6   3   7 
[1]   0   8   4   9   2 (10)  5   1   6   3   7 
[2]   0   8   4   9   2  10   5 (11)  1   6   3   7 
[3]   0   8   4   9   2  10   5  11   1 (12)  6   3   7 
[4]   0   8   4   9   2  10   5  11   1  12   6 (13)  3   7 
[5]   0   8   4   9   2  10   5  11   1  12   6  13   3 (14)  7 
[6]   0   8   4   9   2  10   5  11   1  12   6  13   3  14   7 (15)
[7]   0 (16)  8   4   9   2  10   5  11   1  12   6  13   3  14   7  15 
[8]   0  16   8 (17)  4   9   2  10   5  11   1  12   6  13   3  14   7  15 
[9]   0  16   8  17   4 (18)  9   2  10   5  11   1  12   6  13   3  14   7  15 
[1]   0  16   8  17   4  18   9 (19)  2  10   5  11   1  12   6  13   3  14   7  15 
[2]   0  16   8  17   4  18   9  19   2 (20) 10   5  11   1  12   6  

In [3]:
tests = [
  ('9 players; last marble is worth 25 points', 32),
  ('10 players; last marble is worth 1618 points', 8317),
  ('13 players; last marble is worth 7999 points', 146373),
  ('17 players; last marble is worth 1104 points', 2764),
  ('21 players; last marble is worth 6111 points', 54718),
  ('30 players; last marble is worth 5807 points', 37305)
]

for test,expected in tests:
  actual=score(test)[0]
  assert actual==expected, f'Expected "{expected}", got "{actual}", condition: "{test}"'

In [4]:
puzzle_input = '452 players; last marble is worth 71250 points'
best_score, best_player = score(puzzle_input)
assert best_score, best_player == 388844
best_score, best_player

(388844, 197)

**Part 1 correct answer:** `388844`

## Part 2: Best player's score if the last marble had 100x bigger number

In [5]:
class Marble:
  def __init__(self, number, left, right):
    self.number = number
    self.left = left
    self.right = right

def score_optimized(text, debug=False):
  n_players,last_marble = parse(text)
  current = Marble(0,None,None)
  current.left = current
  current.right = current
  root = current
  player_scores = [0]*n_players
  for marble in range(1, last_marble+1):
    player_i = (marble-1)%n_players # The number (starting with 0) of the player that makes the move with `marble`
    if marble%23 > 0:
      before,after = current.right, current.right.right      
      current = Marble(marble,before,after)
      before.right, after.left = current, current
    else: # A winning turn for the curent player
      popped = current.left.left.left.left.left.left.left
      player_scores[player_i] += popped.number + marble
      before, after = popped.left, popped.right
      before.right, after.left = after, before
      current = after

    if debug:
      marble_i = root
      a = []
      while marble_i is not None:
        mn = str(marble_i.number).rjust(2)
        a.append(f'({mn})' if marble_i==current else f' {mn} ')
        marble_i = marble_i.right if marble_i.right!=root else None

      print(f'[{player_i+1}] {"".join(a)}')

  return max(player_scores)

score_optimized(test_in, True)

[1]   0 ( 1)
[2]   0 ( 2)  1 
[3]   0   2   1 ( 3)
[4]   0 ( 4)  2   1   3 
[5]   0   4   2 ( 5)  1   3 
[6]   0   4   2   5   1 ( 6)  3 
[7]   0   4   2   5   1   6   3 ( 7)
[8]   0 ( 8)  4   2   5   1   6   3   7 
[9]   0   8   4 ( 9)  2   5   1   6   3   7 
[1]   0   8   4   9   2 (10)  5   1   6   3   7 
[2]   0   8   4   9   2  10   5 (11)  1   6   3   7 
[3]   0   8   4   9   2  10   5  11   1 (12)  6   3   7 
[4]   0   8   4   9   2  10   5  11   1  12   6 (13)  3   7 
[5]   0   8   4   9   2  10   5  11   1  12   6  13   3 (14)  7 
[6]   0   8   4   9   2  10   5  11   1  12   6  13   3  14   7 (15)
[7]   0 (16)  8   4   9   2  10   5  11   1  12   6  13   3  14   7  15 
[8]   0  16   8 (17)  4   9   2  10   5  11   1  12   6  13   3  14   7  15 
[9]   0  16   8  17   4 (18)  9   2  10   5  11   1  12   6  13   3  14   7  15 
[1]   0  16   8  17   4  18   9 (19)  2  10   5  11   1  12   6  13   3  14   7  15 
[2]   0  16   8  17   4  18   9  19   2 (20) 10   5  11   1  12   6  

32

In [6]:
for test,expected in tests:
  actual=score_optimized(test)
  assert actual==expected, f'Expected "{expected}", got "{actual}", condition: "{test}"'

In [7]:
part_2_input = '452 players; last marble is worth 7125000 points'
score_optimized(part_2_input)

3212081616

**Incorrect answers**
* `3474683694` is too high.

**Correct answer:** `3212081616`